In [1]:
import napari
import numpy as np
import matplotlib.pyplot as plt
import z5py
from pathlib import Path

In [2]:
def print_key_tree(f: z5py.File):
    print(f"Key structure of z5 file {f.filename}")
    f.visititems(lambda name, obj: print(name))


def read_volume(f: z5py.File, key: str, roi: np.lib.index_tricks.IndexExpression = np.s_[:]):
    try:
        ds = f[key]
    except KeyError:
        print(f"No key {key} in file {f.filename}")
        print_key_tree(f)
        return None
        
    ds.n_threads = 8
    print(f"Reading roi {roi} of volume {key} from {f.filename}")
    vol = ds[roi]
    print(f"Read volume with shape {vol.shape}, data type {vol.dtype}")
    
    return vol

## View n5 datasets

In [3]:
scratch_dir = Path("/scratch/buglakova")
predictions_dir = scratch_dir / "F107_bin2_619-639_predictions"

In [4]:
multicut_n5_path = predictions_dir / "filter_size_235:300.n5"
multicut_n5_path = predictions_dir / "full_multicut_beta_corrected.n5"

In [5]:
f_multicut = z5py.File(multicut_n5_path, "r")

## Read volumes

In [6]:
roi = np.s_[200:250]

In [7]:
raw = read_volume(f_multicut, "raw/raw", roi)
multicut = read_volume(f_multicut, "segmentation/multicut", roi)
filtered = read_volume(f_multicut, "segmentation/filtered", roi)
multicut_init = read_volume(f_multicut, "segmentation/multicut_initial", roi)

Reading roi slice(200, 250, None) of volume raw/raw from /scratch/buglakova/F107_bin2_619-639_predictions/full_multicut_beta_corrected.n5
Read volume with shape (50, 1506, 1647), data type uint8
Reading roi slice(200, 250, None) of volume segmentation/multicut from /scratch/buglakova/F107_bin2_619-639_predictions/full_multicut_beta_corrected.n5
Read volume with shape (50, 1506, 1647), data type uint32
Reading roi slice(200, 250, None) of volume segmentation/filtered from /scratch/buglakova/F107_bin2_619-639_predictions/full_multicut_beta_corrected.n5
Read volume with shape (50, 1506, 1647), data type uint32
Reading roi slice(200, 250, None) of volume segmentation/multicut_initial from /scratch/buglakova/F107_bin2_619-639_predictions/full_multicut_beta_corrected.n5
Read volume with shape (50, 1506, 1647), data type uint32


In [11]:
print_key_tree(f_multicut)

Key structure of z5 file /scratch/buglakova/F107_bin2_619-639_predictions/full_multicut_beta_corrected.n5
corrected_annotation
raw
raw/raw
segmentation
segmentation/multicut
segmentation/multicut_initial
segmentation/multicut_0.6_postprocessed
segmentation/filtered


In [18]:
boundaries = read_volume(f_multicut, "segmentation/boundaries", roi)

No key segmentation/boundaries in file /scratch/buglakova/F107_bin2_619-639_predictions/full_multicut_beta_corrected.n5
Key structure of z5 file /scratch/buglakova/F107_bin2_619-639_predictions/full_multicut_beta_corrected.n5
corrected_annotation
raw
raw/raw
segmentation
segmentation/multicut
segmentation/multicut_initial
segmentation/multicut_0.6_postprocessed
segmentation/filtered


## Open it in napari

In [8]:
v = napari.Viewer()

In [9]:
v.add_image(raw)

<Image layer 'raw' at 0x7f05ac3775e0>

In [10]:
v.add_labels(multicut)

<Labels layer 'multicut' at 0x7f05ac2ab1c0>

In [12]:
v.add_labels(multicut_init)

<Labels layer 'multicut_init' at 0x7f05ac1ed0f0>

In [11]:
v.add_labels(filtered)

<Labels layer 'filtered' at 0x7f05ac32b8b0>

In [17]:
v.add_labels(boundaries)

TypeError: 'NoneType' object is not iterable

In [22]:
raw.shape

(1350, 1506, 1647)